In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))
 
#여백 줄이기
display(HTML("<style>.prompt { min-width: 1ex !important; }</style>"))

#In 부분 없애기
#display(HTML("<style>.prompt bdi { display:none !important; }</style>"))
 
#아예 다 없애기
#display(HTML("<style>.prompt { display:none !important; }</style>"))

/tmp/ipykernel_2595/3328681235.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
traditional_prompt = """
hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book_stay: 1, 2, 3, etc.
hotel-book_day: monday, tuesday, etc.
hotel-book_people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no
train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book_people: 1, 2, 3, etc.
train-leaveat: 20:24, 12:06, etc.
train-arriveby: 05:51, 20:52, etc.
attraction-name: abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area: dontcare, centre, east, north, south, west
attraction-type: architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum,
nightclub, park, special, swimming pool, theatre
restaurant-name: pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop,
etc.
restaurant-food: italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange: dontcare, cheap, moderate, expensive
restaurant-area: centre, east, north, south, west
restaurant-book_time: 13:30, 17:11, etc.
restaurant-book_day: wednesday, friday, etc.
restaurant-book_people: 1, 2, 3, etc.
taxi-destination: copper kettle, magdalene college, lovell lodge
taxi-departure: royal spice, university arms hotel, da vinci pizzeria
taxi-leaveat: 14:45, 11:15, etc.
taxi-arriveby: 15:30, 12:45, etc.

-- the examples below show tracking of slots changes in multi-turn conversational questions for the ontology provided above
"""

slot_classify_prompt = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
All domain-slots have their own number that represent themselves in examples.

domain-slots (number): possible values
hotel-name (0): a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange (1): dontcare, cheap, moderate, expensive
hotel-type (2): hotel, guest house
hotel-parking (3): dontcare, yes, no
hotel-book stay (4): 1, 2, 3, etc.
hotel-book day (5): monday, tuesday, etc.
hotel-book people (6): 1, 2, 3, etc.
hotel-area (7): dontcare, centre, east, north, south, west
hotel-stars (8): dontcare, 0, 1, 2, 3, 4, 5
hotel-internet (9): dontcare, yes, no
train-destination (10): london kings cross, cambridge, peterborough, etc.
train-departure (11): cambridge, stansted airport, etc.
train-day (12): monday, saturday, etc.
train-book people (13): 1, 2, 3, etc.
train-leaveat (14): 20:24, 12:06, etc.
train-arriveby (15): 05:51, 20:52, etc.
attraction-name (16): abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area (17): dontcare, centre, east, north, south, west
attraction-type (18): architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre
restaurant-name (19): pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food (20): italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange (21): dontcare, cheap, moderate, expensive
restaurant-area (22): centre, east, north, south, west
restaurant-book time (23): 13:30, 17:11, etc.
restaurant-book day (24): wednesday, friday, etc.
restaurant-book people (25): 1, 2, 3, etc.
taxi-destination (26): copper kettle, magdalene college, lovell lodge
taxi-departure (27): royal spice, university arms hotel, da vinci pizzeria
taxi-leaveat (28): 14:45, 11:15, etc.
taxi-arriveby (29): 15:30, 12:45, etc.
"""

def our_pred_parse(pred):

    # fix for no states
    if pred == "":
        return {}

    pred_slot_values = {}

    slot_value = pred.split(",")

    value_assigner = "="
    for i in slot_value:
      if value_assigner not in i:
        continue
      else:
        pred_slot_values[i.split(value_assigner)[0].strip()] = i.split(value_assigner)[1].strip()

    return pred_slot_values

def slot_classify_parse(pred):

    # # the output format is "(slot_name = value)"
    # start_pos = pred.rfind("(") + 1
    # end_pos = pred.rfind(")")
    # pred = pred[start_pos:end_pos]

    # fix for no states
    if pred == "":
        return {}

    pred_slot_values = {}

    slot_value = pred.split(",")

    value_assigner = "="
    for i in slot_value:
      i = i.replace("(","").replace(")","")
      if value_assigner not in i:
        continue
      else:
        print(i)
        pred_slot = idx_to_slot(int(i.split(value_assigner)[0].strip()))
        pred_value = i.split(value_assigner)[1].strip()
        print(pred_slot + " " + pred_value)
        pred_slot_values[pred_slot] = pred_value

    return pred_slot_values

slot_idx_dict = {'hotel-name': 0,
 'hotel-pricerange': 1,
 'hotel-type': 2,
 'hotel-parking': 3,
 'hotel-book stay': 4,
 'hotel-book day': 5,
 'hotel-book people': 6,
 'hotel-area': 7,
 'hotel-stars': 8,
 'hotel-internet': 9,
 'train-destination': 10,
 'train-departure': 11,
 'train-day': 12,
 'train-book people': 13,
 'train-leaveat': 14,
 'train-arriveby': 15,
 'attraction-name': 16,
 'attraction-area': 17,
 'attraction-type': 18,
 'restaurant-name': 19,
 'restaurant-food': 20,
 'restaurant-pricerange': 21,
 'restaurant-area': 22,
 'restaurant-book time': 23,
 'restaurant-book day': 24,
 'restaurant-book people': 25,
 'taxi-destination': 26,
 'taxi-departure': 27,
 'taxi-leaveat': 28,
 'taxi-arriveby': 29}

def slot_to_idx(str):
    # str = str.replace(" ","_")
    if str in slot_idx_dict.keys():
        return slot_idx_dict[str]
    else:
        return -1

def idx_to_slot(idx):
    if idx in slot_idx_dict.values():
        return {v: k for k, v in slot_idx_dict.items()}[idx]
    else:
        return "invalid_slot_idx"
    
def conversion(prompt, reverse=False):
    conversion_dict = {"leaveat": "depart_time", "arriveby": "arrive_by_time",
                       "book_stay": "book_number_of_days",
                       "food": "food_type"}
    reverse_conversion_dict = {v: k for k, v in conversion_dict.items()}
    used_dict = reverse_conversion_dict if reverse else conversion_dict

    for k, v in used_dict.items():
        prompt = prompt.replace(k, v)
    return prompt

slot_classify_parse("Corresponding values: hotel-name = a and b guest house, attraction-type = museum, hotel-book stay = 4, hotel-book day = tuesday, hotel-book people = 6 this is the 57th example. PMUL2636.json_turn_5")

Corresponding values: hotel-name = a and b guest house


ValueError: invalid literal for int() with base 10: 'Corresponding values: hotel-name'

In [3]:
import json
import time
from api_request.gpt35_turbo_completion import gpt35_turbo_completion

running_log_path = "expts/230419_1637-gpt35_turbo_5p_v2_slot_classify_with_desc_0to368/running_log.json"
with open(running_log_path,'r') as f:
    content = json.load(f)
            
# for idx, i in enumerate(content):
#     if i["pred_status"] == "wrong":
#         if i["predicted_slot_values"] != i["turn_slot_values"]:
#             print(idx)
#             print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
#             print("user uttr  ==== " + i["dialog"]["usr"][-1])
#             print()
#             print("prev_label ==== " + str(i["last_slot_values"]))
#             print()
#             print("completion ==== " + i["completion"])
#             print()
#             print("prediction ==== " + str(i["predicted_slot_values"]))
#             print("labeling.. ==== " + str(i["turn_slot_values"]))
#             print("\n\n\n\n\n\n")
            
for idx, i in enumerate(content):
    if i["pred_status"] == "wrong":
        if i["predicted_slot_values"] != i["turn_slot_values"]:
            set_predicted = {f'{k} = {v}' for k,v in i["predicted_slot_values"].items()}
            set_label = {f'{k} = {v}' for k,v in i["turn_slot_values"].items()}
            # subset일 경우 (turn_slot_values에 있는 값을 빠뜨렸을 경우)
            follow_up_prompt = ""
            if set_predicted.issubset(set_label):
                print(idx)
                print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
                print("user uttr  ==== " + i["dialog"]["usr"][-1])
                print()
                print("prev_label ==== " + str(i["last_slot_values"]))
                print()
                print("completion ==== " + i["completion"])
                print()
                print("prediction ==== " + str(i["predicted_slot_values"]))
                print("labeling.. ==== " + str(i["turn_slot_values"]))
                print("\n\n")
                
                follow_up_prompt += f"example\n"
    
                last_slot_values = {s: v.split(
                '|')[0] for s, v in i['last_slot_values'].items()}
                follow_up_prompt += f"[context] {conversion(', '.join({f'{slot} = {value}' for slot, value in last_slot_values.items()}))}\n"
            
                last_sys_utt = i['dialog']['sys'][-1]
                if last_sys_utt == 'none':
                    last_sys_utt = ''
                follow_up_prompt += f"[system] {last_sys_utt}\n"
                follow_up_prompt += f"[user] {i['dialog']['usr'][-1]}\n"
                
                follow_up_prompt += f"\nIn the example above, [context] is dialogue states that represent previous state of dialogue turns.\n[system] and [user] are a current system utterance and user utterance respectively.\nQ: what is the intent of user?"
                
                completion = gpt35_turbo_completion(follow_up_prompt)
                
                time.sleep(1)
                
                follow_up_prompt += f"\nA: {completion}"
                follow_up_prompt += f"\nQ: then, Is there any implicit information between [user] and [system] that we need to consider when answering the question?"
                follow_up_prompt += f"\nA:"
                
                print("************* prompt **************")
                print(follow_up_prompt)
                
                completion2 = gpt35_turbo_completion(follow_up_prompt)
                print("\n************* completion **************")
                print(completion2)
                
                time.sleep(1)
                
                print("\n\n\n")

6
sys  uttr  ==== could you double check that you've spelled the name correctly ? the closest i can find is nandos .
user uttr  ==== it s not a restaurant , it s an attraction . nusha .

prev_label ==== {'restaurant-name': 'nusha'}

completion ==== 16 = nusha

prediction ==== {'attraction-name': 'nusha'}
labeling.. ==== {'attraction-name': 'nusha', 'restaurant-name': '[DELETE]'}



************* prompt **************
example
[context] restaurant-name = nusha
[system] could you double check that you've spelled the name correctly ? the closest i can find is nandos .
[user] it s not a restaurant , it s an attraction . nusha .

In the example above, [context] is dialogue states that represent previous state of dialogue turns.
[system] and [user] are a current system utterance and user utterance respectively.
Q: what is the intent of user?
A: A: The intent of the user is to correct the system's misunderstanding and clarify that "nusha" is not a restaurant but an attraction.
Q: then, Is ther

KeyboardInterrupt: 

In [4]:
import json
import time
from api_request.gpt35_turbo_completion import gpt35_turbo_completion

running_log_path = "expts/230508_1930-gpt35_turbo_5p_v2_baseline_blank_blanket_0to100/running_log.json"
with open(running_log_path,'r') as f:
    content = json.load(f)
    
for idx, i in enumerate(content):
#     if i["pred_status"] == "wrong":
#         if i["predicted_slot_values"] != i["turn_slot_values"]:
    print(idx)
    print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
    print("user uttr  ==== " + i["dialog"]["usr"][-1])
    print()
    print("prev_label ==== " + str(i["last_slot_values"]))
    print()
    print(i["prompt"])
    print()
    print("completion ==== " + i["completion"])
    print()
    print("prediction ==== " + str(i["predicted_slot_values"]))
    print("labeling.. ==== " + str(i["turn_slot_values"]))
    print("\n\n\n\n\n\n")

# for idx, i in enumerate(content):
#     if i["pred_status"] == "wrong":
#         if i["predicted_slot_values"] != i["turn_slot_values"]:
#             print(idx)
#             print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
#             print("user uttr  ==== " + i["dialog"]["usr"][-1])
#             print()
#             print("prev_label ==== " + str(i["last_slot_values"]))
#             print()
#             print(i["prompt"])
#             print()
#             print("completion ==== " + i["completion"])
#             print()
#             print("prediction ==== " + str(i["predicted_slot_values"]))
#             print("labeling.. ==== " + str(i["turn_slot_values"]))
#             print("\n\n\n\n\n\n")

0
sys  uttr  ==== 
user uttr  ==== i would like a taxi from saint john s college to pizza hut fen ditton .

prev_label ==== {}

Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
you don't have to find other changed domain-slots if they are not in the list.

hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book stay: 1, 2, 3, etc.
hotel-book day: monday, tuesday, etc.
hotel-book people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
trai